In [4]:
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 explode, col

# UDF(사용자 정의 함수) 작업을 위한 SparkSession 생성
spark = (SparkSession.builder  # SparkSession 빌더 패턴 시작
         .appName("write-udfs")  # UDF 작업 애플리케이션
         .master("spark://spark-master:7077")  # Spark 마스터 연결
         .config("spark.executor.memory", "512m")  # 실행자 메모리 설정
         .getOrCreate())  # 세션 생성 또는 기존 세션 사용

# 로그 레벨을 ERROR로 설정
spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

In [2]:
# UDF 예제를 위한 노벨상 데이터 로드
df = (spark.read.format("json")  # JSON 형식 지정
      .option("multiLine", "true")  # 여러 줄 JSON 처리
      .load("../data/nobel_prizes.json"))  # 노벨상 데이터 파일 로드

In [5]:
# UDF 예제를 위한 데이터 전처리 - 중첩 구조 평면화 및 필터링
df_flattened = (
    df
    .withColumn("laureates",explode(col("laureates")))  # 수상자 배열을 개별 행으로 분해
    .select(col("category")  # 카테고리 선택
            ,col("year")  # 연도 선택
            ,col("overallMotivation")  # 전체 동기 선택
            ,col("laureates.id")  # 수상자 ID 선택
            ,col("laureates.firstname")  # 수상자 이름 선택
            ,col("laureates.surname")  # 수상자 성 선택
            ,col("laureates.share")  # 수상 비율 선택
            ,col("laureates.motivation"))  # 수상 이유 선택
    .filter(col("laureates.firstname").isNotNull() & col("laureates.surname").isNotNull()))  # 이름과 성이 모두 있는 데이터만 필터링

In [6]:
# 사용자 정의 함수(UDF) 정의 - 이름과 성을 결합하는 함수
def concat(  # 문자열 연결first_name, last_name):
    """
    이름과 성을 겵백으로 구분하여 결합하는 함수
    """
    return first_name + " " + last_name

In [7]:
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 udf
# Python 함수를 Spark UDF로 변환 - 기본 방식(반환 타입 자동 추론)
concat_udf = udf(  # 사용자 정의 함수 생성concat)

In [8]:
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 StringType
# 명시적 반환 타입을 지정한 UDF 생성 - 성능 최적화를 위해 권장
concat_udf = udf(  # 사용자 정의 함수 생성concat, StringType())

In [9]:
# UDF를 사용하여 새로운 컴럼 추가 - 이름과 성을 결합한 전체 이름
df_flattened = df_flattened.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"full_name", concat_udf(df_flattened["firstname"], df_flattened["surname"]))

In [10]:
# UDF가 적용된 DataFrame 출력 - full_name 컴럼 확인
df_flattened.show()  # DataFrame 내용 출력

+----------+----+--------------------+----+----------+-----------+-----+--------------------+-----------------+
|  category|year|   overallMotivation|  id| firstname|    surname|share|          motivation|        full_name|
+----------+----+--------------------+----+----------+-----------+-----+--------------------+-----------------+
| chemistry|2022|                null|1015|   Carolyn|   Bertozzi|    3|"for the developm...| Carolyn Bertozzi|
| chemistry|2022|                null|1016|    Morten|     Meldal|    3|"for the developm...|    Morten Meldal|
| chemistry|2022|                null| 743|     Barry|  Sharpless|    3|"for the developm...|  Barry Sharpless|
| economics|2022|                null|1021|       Ben|   Bernanke|    3|"for research on ...|     Ben Bernanke|
| economics|2022|                null|1022|   Douglas|    Diamond|    3|"for research on ...|  Douglas Diamond|
| economics|2022|                null|1023|    Philip|     Dybvig|    3|"for research on ...|    Philip 

### Using UDFs in Spark SQL

In [11]:
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 udf
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 IntegerType

# Spark SQL에서 사용할 UDF 정의 - 숫자의 제곱 계산
def square_udf(  # 사용자 정의 함수 생성x):
    """
    입력값의 제곱을 반환하는 함수
    """
    return x ** 2

# UDF를 Spark SQL에서 사용할 수 있도록 등록
spark.udf.register(  # UDF를 SQL에서 사용할 수 있도록 등록"square", square_udf, IntegerType())

# 테스트용 DataFrame 생성
df = spark.createDataFrame([(1,), (2,), (3,), (4,), (5,)], ["num"])

# DataFrame을 임시 테이블로 등록하여 SQL 쿼리에서 사용 가능하게 설정
df.createOrReplaceTempView(  # 임시 뷰 생성"numbers")
# 등록된 UDF를 SQL 쿼리에서 사용
result = spark.sql(  # SQL 쿼리 실행"SELECT num, square(num) AS square_num FROM numbers")

# SQL UDF 사용 결과 출력
result.show()  # DataFrame 내용 출력

+---+----------+
|num|square_num|
+---+----------+
|  1|         1|
|  2|         4|
|  3|         9|
|  4|        16|
|  5|        25|
+---+----------+



In [12]:
# Spark 세션 종료 - 리소스 정리
spark.stop()  # Spark 세션 종료 - 리소스 정리